# Compendio de Optimización No Lineal:
## Teoría Matemática, Análisis de Convergencia y Algoritmos Computacionales

**Presentado por:** Ing. Melvin E. Padilla  
**Cátedra:** Programación Matemática  
**Maestría en Matemática y Computación** 
**Universidad de Carabobo, Venezuela** 
**2025**

---

### Resumen
Este documento constituye una recopilación exhaustiva sobre la optimización matemática no lineal sin restricciones. Integra los fundamentos topológicos (convexidad, concavidad), el análisis riguroso de convergencia, el Teorema de Taylor multivariable, la teoría de formas cuadráticas y la implementación computacional en Python. Está diseñado para cubrir desde la teoría base hasta la aplicación numérica avanzada necesaria en ingeniería.

### Índice

1. Introducción y Marco de Referencia
2. El Problema Unidimensional
   - 2.1. Topología: Unimodalidad, Convexidad y Concavidad
   - 2.1.1. Implementación Computacional: Visualización de Dualidad
   - 2.2. Caracterización de Óptimos Locales
   - 2.3. Métodos de Búsqueda sin Derivadas
     - 2.3.1. Método de la Sección Dorada
     - 2.3.2. Interpolación Parabólica
3. El Problema Multivariable: Fundamentos Matemáticos
   - 3.1. El Gradiente y la Hessiana
   - 3.2. Teorema de Taylor Multivariable
   - 3.3. Formas Cuadráticas y Criterio de Sylvester
   - 3.4. Gráficos de Contorno
4. Algoritmos de Descenso Multivariable
   - 4.1. Análisis de Convergencia
   - 4.2. Método del Máximo Descenso (Gradient Descent)
   - 4.3. Método de Newton Multivariable
5. El Problema del Valle Curvo
   - 5.1. Por qué falla el Gradiente (Primer Orden)
6. La Solución de Newton (Segundo Orden)
   - 6.1. ¿Qué hace la Hessiana realmente?

## 1. Introducción y Marco de Referencia

> **Nota de Cátedra:** El ingeniero de postgrado debe transitar fluidamente entre la abstracción matemática y la implementación numérica. Este documento no es un manual de usuario, sino un compendio teórico-práctico. Las referencias citadas a continuación son la base canónica sobre la cual se construyen solvers comerciales como los de GAMS, MATLAB o Python SciPy.

El problema de optimización busca encontrar un vector de variables de diseño que minimice o maximice una función objetivo, sujeto a restricciones o libre de ellas. En ingeniería, esto es fundamental para el diseño de sistemas eficientes, desde estructuras mecánicas hasta procesos químicos.

Para el desarrollo riguroso de este contenido, se han tomado como base las siguientes referencias canónicas en el campo de la optimización numérica:

1. Nocedal, J., & Wright, S. J. (2006). *Numerical Optimization*. Springer.
2. Rao, S. S. (2009). *Engineering Optimization: Theory and Practice*. John Wiley & Sons.
3. Luenberger, D. G., & Ye, Y. (2008). *Linear and Nonlinear Programming*. Springer.

## 2. El Problema Unidimensional

El problema unidimensional busca encontrar un escalar $x^*$ tal que optimice $f(x)$, donde $f: \mathbb{R} \to \mathbb{R}$. Aunque los problemas reales suelen ser multivariables, la optimización unidimensional es crítica porque forma la base de los algoritmos de “Búsqueda de Línea” (Line Search) utilizados en cada iteración de los problemas de n-dimensiones.

### 2.1. Topología: Unimodalidad, Convexidad y Concavidad

> **Nota de Cátedra:** La dualidad es clave en ingeniería: Maximizar la eficiencia (cóncava) es equivalente a minimizar el desperdicio (convexa). Matemáticamente, maximizar $f(x)$ es idéntico a minimizar $-f(x)$. Entender esta simetría permite usar los mismos algoritmos para ambos propósitos.

Para garantizar la existencia y unicidad de soluciones, debemos analizar la topología de la función.

**Definición 2.1 (Unimodalidad).** Una función $f(x)$ es unimodal en el intervalo $[a, b]$ si existe un único $x^* \in [a, b]$ tal que $f(x)$ es monótona decreciente a la izquierda de $x^*$ y creciente a la derecha (para el caso de minimización). Es la condición mínima necesaria para que los métodos de reducción de intervalo funcionen.

**Definición 2.2 (Convexidad).** Una función $f$ es convexa si el segmento de línea que une dos puntos cualesquiera de la gráfica queda por encima de la curva. Matemáticamente, para todo $x, y$ en el dominio y $\lambda \in [0, 1]$:
$$f(\lambda x + (1 - \lambda)y) \leq \lambda f(x) + (1 - \lambda)f(y)$$
Si $f$ es dos veces diferenciable ($f \in C^2$), la convexidad implica $f''(x) \geq 0$ en todo el dominio. Implicación: Cualquier mínimo local encontrado en una función convexa es garantizado ser un Mínimo Global.

**Definición 2.3 (Concavidad).** Una función $f$ es cóncava si el segmento de línea que une dos puntos cualesquiera queda por debajo de la curva (forma de colina o tazón invertido). Si $f \in C^2$, implica $f''(x) \leq 0$. Implicación: Cualquier máximo local es un Máximo Global.

#### 2.1.1. Implementación Computacional: Visualización de Dualidad
El siguiente script en Python utiliza matplotlib para graficar ambas topologías, permitiendo visualizar la relación entre el signo de la segunda derivada y la forma de la función.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Definir rango de evaluacion
x = np.linspace(-10, 10, 400)

# Funcion Convexa (Tazon) -> f''(x) > 0 -> Para Minimizar
# Ejemplo: Energia Potencial
f_convex = x**2

# Funcion Concava (Colina) -> f''(x) < 0 -> Para Maximizar
# Ejemplo: Curva de Rendimiento
f_concave = -x**2 + 100

plt.figure(figsize=(10, 5))

# Grafica 1: Convexidad
plt.subplot(1, 2, 1)
plt.plot(x, f_convex, 'b', label=r'Convexa ($x^2$)')
plt.title('Convexidad (Busqueda de Minimo)')
plt.grid(True)
plt.legend()

# Grafica 2: Concavidad
plt.subplot(1, 2, 2)
plt.plot(x, f_concave, 'r', label=r'Concava ($-x^2 + 100$)')
plt.title('Concavidad (Busqueda de Maximo)')
plt.grid(True)
plt.legend()

plt.show()

### 2.2. Caracterización de Óptimos Locales

Para funciones suaves, el cálculo diferencial nos ofrece herramientas poderosas mediante la expansión de Taylor alrededor de un punto $x^*$:
$$f(x) = f(x^*) + f'(x^*)(x - x^*) + \frac{1}{2}f''(x^*)(x - x^*)^2 + O((x - x^*)^3)$$

1. **Condición Necesaria (Primer Orden):** Si $x^*$ es un óptimo local, entonces $f'(x^*) = 0$ (punto estacionario).
2. **Condición Suficiente (Segundo Orden):**
   - Si $f'(x^*) = 0$ y $f''(x^*) > 0$, entonces $x^*$ es un Mínimo Local Estricto.
   - Si $f'(x^*) = 0$ y $f''(x^*) < 0$, entonces $x^*$ es un Máximo Local Estricto.
   - Si $f''(x^*) = 0$, la prueba no es concluyente (puede ser punto de inflexión).

### 2.3. Métodos de Búsqueda sin Derivadas

> **Nota de Cátedra:** En la práctica ingenieril, muchas veces no tenemos una fórmula bonita $f(x)$. Tenemos una simulación de “caja negra” (ej. Aspen HYSYS, ANSYS). En estos casos, no podemos calcular derivadas analíticas ($f'(x)$). Recurrimos a métodos geométricos.

#### 2.3.1. Método de la Sección Dorada
Es un método de reducción de intervalos que no requiere derivadas. Se basa en reducir el intervalo de incertidumbre $[a, b]$ evaluando la función en dos puntos intermedios determinados por la razón áurea $\phi = \frac{1+\sqrt{5}}{2} \approx 1.618$.

**Ventaja:** Robusto. Garantiza convergencia si la función es unimodal.
**Tasa de Convergencia:** Lineal (reducción del intervalo por factor 0.618 en cada paso).

**Ejemplo de Ingeniería 2.1 (Optimización de Reactores Químicos).** Se desea encontrar la temperatura $T$ que maximiza el rendimiento $R(T)$ de una reacción. La función $R(T)$ proviene de una simulación compleja. La Sección Dorada acota el rango sistemáticamente sin necesitar gradientes termodinámicos.

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
from typing import Callable, Tuple, List

def golden_section_search_visual(
    f: Callable[[float], float],
    a: float,
    b: float,
    tol: float = 1e-5
) -> Tuple[float, int, List[Tuple[float, float]]]:
    """
    Encuentra el mínimo de f en [a , b ] usando la Sección Dorada .
    Retorna : ( punto_minimo , iteraciones , historial_intervalos )
    """
    # La razón áurea ( phi ) inversa : ( sqrt (5) -1) /2 approx 0.618
    ratio = (math.sqrt(5) - 1) / 2
    
    # Inicialización
    x1 = b - ratio * (b - a)
    x2 = a + ratio * (b - a)
    f1, f2 = f(x1), f(x2)
    
    # Guardamos la historia : (a , b )
    history = [(a, b)]
    iters = 0
    
    while (b - a) > tol:
        iters += 1
        if f1 < f2:
            # El mínimo está en el sub-intervalo izquierdo
            b = x2
            x2 = x1
            f2 = f1
            x1 = b - ratio * (b - a)
            f1 = f(x1)
        else:
            # El mínimo está en el sub-intervalo derecho
            a = x1
            x1 = x2
            f1 = f2
            x2 = a + ratio * (b - a)
            f2 = f(x2)
            
        history.append((a, b))
        
    return (a + b) / 2, iters, history

def plot_results(f, a_orig, b_orig, min_x, history):
    """ Genera gráficos explicativos del proceso de optimización """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # --- GRÁFICO 1: La Función y el Mínimo ---
    x = np.linspace(a_orig, b_orig, 400)
    y = [f(val) for val in x]
    
    ax1.plot(x, y, label='f(x)', color='blue', alpha=0.6)
    ax1.scatter(min_x, f(min_x), color='red', zorder=5, s=100,
                label=f'Mínimo: {min_x:.4f}')
    ax1.set_title('Función Objetivo y Mínimo Encontrado')
    ax1.set_xlabel('x')
    ax1.set_ylabel('f(x)')
    ax1.legend()
    ax1.grid(True, linestyle='--', alpha=0.5)
    
    # --- GRÁFICO 2: Reducción del Intervalo (El "Embudo") ---
    iterations = range(len(history))
    # Calculamos el centro y el ancho de cada intervalo para graficarlo
    centers = [(h[0] + h[1])/2 for h in history]
    errors = [(h[1] - h[0])/2 for h in history]
    
    ax2.errorbar(centers, iterations, xerr=errors, fmt='o',
                 capsize=3, color='green', ecolor='orange', elinewidth=3)
    
    ax2.invert_yaxis()
    ax2.set_title(f'Convergencia del Intervalo (Sección Dorada)')
    ax2.set_xlabel('Espacio de Búsqueda (x)')
    ax2.set_ylabel('Número de Iteración')
    ax2.axvline(x=min_x, color='red', linestyle='--', alpha=0.3,
                label='Mínimo Real')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# --- EJECUCIÓN ---
# 1. Definimos la función lambda
func = lambda x: (x - 4)**2 + 10

# 2. Definimos el intervalo inicial
inicio, fin = 0, 10

# 3. Ejecutamos el algoritmo
resultado, n_iters, historial = golden_section_search_visual(func, inicio, fin)

print(f"--> Óptimo encontrado en x = {resultado:.5f}")
print(f"--> Precisión alcanzada tras {n_iters} iteraciones")
print("--> Generando gráficos...")

# 4. Graficamos
plot_results(func, inicio, fin, resultado, historial)

#### 2.3.2. Interpolación Parabólica
Este método aprovecha que muchas funciones suaves se comportan cuadráticamente cerca del óptimo. Dados tres puntos $(x_1, f_1), (x_2, f_2), (x_3, f_3)$, se ajusta un polinomio de segundo grado $q(x)$. El mínimo de este polinomio se usa como la nueva estimación del óptimo.

$$x_{new} = x_2 - \frac{1}{2} \frac{(x_2 - x_1)^2(f_2 - f_3) - (x_2 - x_3)^2(f_2 - f_1)}{(x_2 - x_1)(f_2 - f_3) - (x_2 - x_3)(f_2 - f_1)}$$

Posee una tasa de convergencia superlineal (orden $\approx 1.324$), siendo más rápida que la Sección Dorada cerca del óptimo, pero menos robusta lejos de él.

## 3. El Problema Multivariable: Fundamentos Matemáticos
Aquí $f : \mathbb{R}^n \to \mathbb{R}$. La variable de diseño es un vector $x = [x_1, x_2, ..., x_n]^T$.

### 3.1. El Gradiente y la Hessiana

> **Nota de Cátedra:** Deben visualizar el Gradiente como su brújula (indica la dirección) y la Hessiana como el mapa de relieve (indica la curvatura del terreno). Sin estas dos definiciones, estamos ciegos en un espacio n-dimensional. En Python, utilizamos la librería `sympy` para cálculo simbólico exacto.

**Definición 3.1 (Vector Gradiente).** Es el vector de primeras derivadas parciales. Físicamente, apunta en la dirección de máximo crecimiento de la función.
$$\nabla f(x) = \left[ \frac{\partial f}{\partial x_1}, ..., \frac{\partial f}{\partial x_n} \right]^T$$
En el óptimo sin restricciones, el gradiente debe ser nulo ($\nabla f = 0$).

**Definición 3.2 (Matriz Hessiana).** Es la matriz simétrica de $n \times n$ de segundas derivadas parciales.
$$H_{ij}(x) = \frac{\partial^2 f}{\partial x_i \partial x_j}$$
Proporciona información sobre la curvatura de la función y es crucial para determinar si un punto estacionario es máximo, mínimo o punto de silla.

In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def analyze_and_plot_multivar(expr_str):
    """
    Analiza una función de dos variables, encuentra su óptimo y genera gráficos 3D/2D.
    """
    # 1. Configuración Simbólica
    x, y = sp.symbols('x y')
    f = sp.sympify(expr_str)
    
    # 2. Cálculo del Gradiente (Primeras derivadas)
    # El gradiente es un vector : [df/dx , df/dy]
    grad_f = sp.Matrix([sp.diff(f, var) for var in (x, y)])
    
    # 3. Cálculo de la Hessiana (Segundas derivadas)
    hessian_f = sp.hessian(f, (x, y))
    
    # 4. Encontrar Puntos Críticos (Donde el gradiente es cero)
    critical_points = sp.solve(grad_f, (x, y))
    
    # Nota: sp.solve puede devolver un diccionario o una lista de tuplas
    if isinstance(critical_points, dict):
        crit_x, crit_y = float(critical_points[x]), float(critical_points[y])
    else:
        # Tomamos el primer punto encontrado para este ejemplo
        crit_x, crit_y = float(critical_points[0][0]), float(critical_points[0][1])
        
    # 5. Evaluar Hessiana en el punto crítico para ver concavidad
    H_val = hessian_f.subs({x: crit_x, y: crit_y})
    eigenvals = H_val.eigenvals() # Clave para saber si es min/max
    
    print(f"--- ANÁLISIS ---")
    print(f"Función: f(x, y) = {f}")
    print(f"Gradiente: {grad_f.T}")
    print(f"Hessiana:\n{hessian_f}")
    print(f"Punto Crítico encontrado: ({crit_x}, {crit_y})")
    print(f"Autovalores de la Hessiana: {list(eigenvals.keys())}")
    
    # --- VISUALIZACIÓN ---
    
    # Convertir función simbólica a función numérica de Python (lambdify)
    f_num = sp.lambdify((x, y), f, 'numpy')
    
    # Crear rejilla de datos (Grid) alrededor del punto crítico
    range_val = 5
    x_vals = np.linspace(crit_x - range_val, crit_x + range_val, 100)
    y_vals = np.linspace(crit_y - range_val, crit_y + range_val, 100)
    X, Y = np.meshgrid(x_vals, y_vals)
    Z = f_num(X, Y)
    
    fig = plt.figure(figsize=(16, 7))
    
    # GRÁFICO 1: Superficie 3D
    ax1 = fig.add_subplot(1, 2, 1, projection='3d')
    surf = ax1.plot_surface(X, Y, Z, cmap='viridis', alpha=0.8, edgecolor='none')
    ax1.scatter(crit_x, crit_y, f_num(crit_x, crit_y), color='red', s=100, label='Punto Crítico')
    ax1.set_title('Superficie 3D: f(x, y)')
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.set_zlabel('f(x, y)')
    fig.colorbar(surf, ax=ax1, shrink=0.5, aspect=5)
    ax1.view_init(elev=30, azim=45)
    
    # GRÁFICO 2: Mapa de Contorno (Vista superior)
    ax2 = fig.add_subplot(1, 2, 2)
    contour = ax2.contourf(X, Y, Z, levels=20, cmap='viridis')
    ax2.scatter(crit_x, crit_y, color='red', s=100, marker='x', label=f'Mínimo ({crit_x},{crit_y})')
    
    # Opcional : Agregar flechas de gradiente (Quiver plot)
    grad_x_func = sp.lambdify((x, y), grad_f[0], 'numpy')
    grad_y_func = sp.lambdify((x, y), grad_f[1], 'numpy')
    skip = (slice(None, None, 10), slice(None, None, 10))
    U = grad_x_func(X, Y)
    V = grad_y_func(X, Y)
    
    # Negativo del gradiente apunta al mínimo
    ax2.quiver(X[skip], Y[skip], -U[skip], -V[skip], color='white', alpha=0.5)
    
    ax2.set_title('Curvas de Nivel y Dirección de Descenso')
    ax2.set_xlabel('x')
    ax2.set_ylabel('y')
    ax2.legend()
    ax2.grid(True, linestyle='--', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Ejecución
# Usamos tu ejemplo: x^2 + y^2 + x*y
expr = "x**2 + y**2 + x*y"
analyze_and_plot_multivar(expr)

### 3.2. Teorema de Taylor Multivariable
Para entender los algoritmos de optimización avanzados (como Newton), debemos comprender la aproximación local de segundo orden. Sea $f : \mathbb{R}^n \to \mathbb{R}$ con derivadas continuas hasta segundo orden. La expansión alrededor de $x$ en dirección $h$ es:
$$f(x + h) = f(x) + \nabla f(x)^T h + \frac{1}{2}h^T H(x)h + o(\lVert h \rVert^2)$$
Esta ecuación justifica que si el gradiente es cero ($\nabla f = 0$), el comportamiento de la función está dominado enteramente por el término cuadrático $\frac{1}{2}h^T H h$.

### 3.3. Formas Cuadráticas y Criterio de Sylvester
Para asegurar que un punto estacionario es un **Mínimo Estricto**, la Hessiana $H$ debe ser **Definida Positiva**. Esto significa que $h^T H h > 0$ para todo vector no nulo $h$.

**Teorema 3.1 (Criterio de Sylvester).** Una matriz simétrica $H$ es definida positiva si y solo si todos sus Menores Principales Dominantes ($\Delta_k$) son estrictamente positivos.

Para una matriz $3 \times 3$:
$$ H = \begin{bmatrix} h_{11} & h_{12} & h_{13} \\ h_{21} & h_{22} & h_{23} \\ h_{31} & h_{32} & h_{33} \end{bmatrix} $$
1. $\Delta_1 = h_{11} > 0$
2. $\Delta_2 = \det \begin{pmatrix} h_{11} & h_{12} \\ h_{21} & h_{22} \end{pmatrix} > 0$
3. $\Delta_3 = \det(H) > 0$

Si se cumple, la función tiene forma de “tazón” y el punto es un mínimo. Si los signos alternan ($\Delta_1 < 0, \Delta_2 > 0, \Delta_3 < 0$), la matriz es definida negativa (Máximo).

### 3.4. Gráficos de Contorno
Un gráfico de contorno representa curvas de nivel $C_k = \{x \in \mathbb{R}^2 | f(x) = k\}$. Son la proyección 2D de la superficie 3D. Propiedad fundamental: El vector gradiente $\nabla f(x)$ es siempre **ortogonal** (perpendicular) a la curva de nivel en el punto $x$.

## 4. Algoritmos de Descenso Multivariable

Casi todos los algoritmos de optimización iterativa multivariable siguen la misma estructura canónica:
$$x_{k+1} = x_k + \alpha_k d_k$$
Donde $d_k$ es la dirección de búsqueda y $\alpha_k$ es el tamaño de paso.

### 4.1. Análisis de Convergencia
> **Nota de Cátedra:** En ingeniería de control o sistemas en tiempo real, no basta con encontrar la solución; importa qué tan rápido llegamos a ella.

Se dice que la convergencia es de orden $p$ si existe $C > 0$ tal que $\lim_{k\to\infty} \frac{|e_{k+1}|}{|e_k|^p} = C$, donde $e_k$ es el error.

- **Convergencia Lineal ($p = 1$):** El error se reduce en una fracción constante. (Ej. Gradiente Descendente). Robusta pero lenta.
- **Convergencia Cuadrática ($p = 2$):** El número de cifras significativas correctas se duplica en cada iteración. (Ej. Método de Newton). Extremadamente rápida cerca del óptimo.

### 4.2. Método del Máximo Descenso (Gradient Descent)
Se elige $d_k = -\nabla f(x_k)$. Basado en que el negativo del gradiente es la dirección de descenso más rápido localmente.

**Problema:** Genera trayectorias en “zigzag” en valles estrechos (funciones mal condicionadas), haciendo la convergencia muy lenta.
**Uso:** Entrenamiento de Redes Neuronales (Deep Learning).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def f(x):
    """ Función objetivo : f(x, y) = x^2 + 5y^2 (Elipse alargada / Valle estrecho) """
    return x[0]**2 + 5*x[1]**2

def grad_f(x):
    """ Gradiente analítico : f = [2x, 10y] """
    return np.array([2*x[0], 10*x[1]])

def gradient_descent(start_point, learning_rate=0.05, n_iters=50, tol=1e-6):
    """
    Ejecuta el descenso del gradiente con historial y criterio de parada.
    """
    path = [start_point]
    x_curr = start_point.copy()
    
    for i in range(n_iters):
        grad = grad_f(x_curr)
        
        # Criterio de parada: Si la norma del gradiente es casi 0, ya llegamos.
        if np.linalg.norm(grad) < tol:
            print(f"Convergió en la iteración {i}")
            break
            
        # Paso de actualización
        x_curr = x_curr - learning_rate * grad
        path.append(x_curr)
        
    return np.array(path)

# --- PARÁMETROS ---
x_start = np.array([8.0, 4.0])
lr = 0.05 # Tasa de aprendizaje
iterations = 50

# Ejecutar optimización
path = gradient_descent(x_start, learning_rate=lr, n_iters=iterations)

# --- VISUALIZACIÓN ---
# Crear rejilla para el gráfico
x_grid = np.linspace(-10, 10, 200)
y_grid = np.linspace(-5, 5, 200)
X, Y = np.meshgrid(x_grid, y_grid)
Z = f([X, Y]) # Aprovechamos broadcasting de numpy

plt.figure(figsize=(10, 8))

# 1. Mapa de colores de fondo (Topografía)
plt.contourf(X, Y, Z, levels=30, cmap='coolwarm', alpha=0.6)

# 2. Líneas de contorno (Curvas de nivel)
contours = plt.contour(X, Y, Z, levels=20, colors='black', linewidths=0.5, alpha=0.8)
plt.clabel(contours, inline=True, fontsize=8)

# 3. Trayectoria del algoritmo
plt.plot(path[:,0], path[:,1], color='white', linewidth=2, linestyle='--', label='Camino de Descenso')
plt.scatter(path[:,0], path[:,1], color='red', s=30, zorder=5) # Puntos intermedios

# 4. Inicio y Fin destacados
plt.scatter(x_start[0], x_start[1], color='lime', s=150, edgecolors='black', label='Inicio', zorder=10)
plt.scatter(path[-1,0], path[-1,1], color='yellow', marker='*', s=250, edgecolors='black', label='Óptimo Local', zorder=10)

# --- MEJORA CRÍTICA ---
# Esto asegura que 1 unidad en X sea visualmente igual a 1 unidad en Y.
# Sin esto, la ortogonalidad no se ve correctamente.
plt.axis('equal')

plt.title(f'Descenso del Gradiente: Zig-Zag en Valles Estrechos\nLearning Rate: {lr} | Pasos: {len(path)-1}')
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.legend()
plt.grid(True, linestyle=':', alpha=0.3)
plt.show()

### 4.3. Método de Newton Multivariable

Se elige $d_k = -[H(x_k)]^{-1} \nabla f(x_k)$. Minimiza la aproximación cuadrática exacta de la función en cada paso. Transforma el espacio para que los contornos elípticos se vuelvan circulares, apuntando directamente al mínimo.

**Ventaja:** Convergencia Cuadrática.
**Desventaja:** Calcular la inversa de la Hessiana $H^{-1}$ es computacionalmente costoso ($O(n^3)$).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm, solve

# --- DEFINICIÓN DEL PROBLEMA ---
def f(x):
    """ Función objetivo : f(x, y) = x^2 + 5y^2 """
    return x[0]**2 + 5*x[1]**2

def grad_f(x):
    """ Gradiente : f = [2x, 10y] """
    return np.array([2*x[0], 10*x[1]])

def hessian_f(x):
    """ Hessiana : H = [[2, 0], [0, 10]] (Constante en cuadráticas) """
    return np.array([[2, 0],
                     [0, 10]])

# --- ALGORITMOS ---

def gradient_descent(start_point, lr=0.05, steps=20):
    """ Descenso de Gradiente (para comparación) """
    path = [start_point]
    x = np.array(start_point)
    for _ in range(steps):
        g = grad_f(x)
        x = x - lr * g
        path.append(x)
    return np.array(path)

def newton_method(start_point, tol=1e-6, max_iter=10):
    """ Método de Newton """
    path = [start_point]
    x = np.array(start_point)
    
    for i in range(max_iter):
        g = grad_f(x)
        H = hessian_f(x)
        
        # Criterio de parada
        if norm(g) < tol:
            print(f"Newton: Convergencia alcanzada en iteración {i}")
            break
            
        # Paso de Newton : H * direction = -g
        # Resolvemos el sistema lineal en lugar de invertir la matriz (más eficiente)
        direction = solve(H, -g)
        x = x + direction
        path.append(x)
        
    return np.array(path)

# --- EJECUCIÓN ---
start = [8.0, 4.0]

# 1. Corremos Gradient Descent (el "lento")
path_gd = gradient_descent(start, lr=0.05, steps=20)

# 2. Corremos Newton (el "rápido")
path_newton = newton_method(start)

print(f"Pasos Gradiente: {len(path_gd)-1}")
print(f"Pasos Newton: {len(path_newton)-1}")

# --- VISUALIZACIÓN ---
x_grid = np.linspace(-10, 10, 200)
y_grid = np.linspace(-5, 5, 200)
X, Y = np.meshgrid(x_grid, y_grid)
Z = f([X, Y])

plt.figure(figsize=(10, 8))

# Contornos
plt.contourf(X, Y, Z, levels=30, cmap='gray_r', alpha=0.4)
plt.contour(X, Y, Z, levels=20, colors='black', linewidths=0.5, alpha=0.5)

# Trayectoria Gradiente (Rojo)
plt.plot(path_gd[:,0], path_gd[:,1], 'r--o', label='Gradiente (Primer Orden)', markersize=4)

# Trayectoria Newton (Azul)
plt.plot(path_newton[:,0], path_newton[:,1], 'b-o', linewidth=3, label='Newton (Segundo Orden)', markersize=8)

# Puntos clave
plt.scatter(start[0], start[1], c='lime', s=150, edgecolors='black', zorder=10, label='Inicio')
plt.scatter(0, 0, c='yellow', marker='*', s=200, edgecolors='black', zorder=10, label='Óptimo')

plt.title('Comparación: Gradiente vs. Newton \n (El poder de la Curvatura)')
plt.xlabel('x')
plt.ylabel('y')
plt.axis('equal')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### Análisis de Convergencia: Gradiente vs. Newton

Si observamos el gráfico generado, notaremos comportamientos radicalmente distintos:

1. **Línea Roja (Gradiente):** Va "tanteando" el camino. Como la elipse es muy alargada, el gradiente le indica que baje muy rápido en el eje Y, pero avanza muy poco en el eje X, provocando el característico efecto de zig-zag. Esto sucede porque el método solo utiliza la pendiente (información de la primera derivada).
2. **Línea Azul (Newton):** Va directo al centro en un solo salto.

**Razón Matemática:**
El método de Newton intenta aproximar la función localmente mediante una parábola (aproximación cuadrática) y salta directamente al mínimo de esa parábola teórica. Dado que tu función original $f(x)$ **YA ES** una parábola (una función cuadrática perfecta), la aproximación es exacta, y el cálculo matemático lleva al mínimo global de manera instantánea.

**El “Truco” de Newton:**
La regla de actualización se define formalmente como $x_{new} = x - H^{-1} \nabla f$. La matriz Hessiana $H$ contiene la información de la curvatura (nos dice "qué tan alargada es la elipse" en cada dirección). Al multiplicar el gradiente por la inversa de la Hessiana ($H^{-1}$), el algoritmo esencialmente "normaliza" el terreno. Desde su perspectiva matemática, convierte la elipse distorsionada en un círculo perfecto, permitiéndole ignorar la distorsión y dar el paso directo hacia el óptimo.

## 5. El Problema del Valle Curvo

La función de Rosenbrock, conocida como la "Función Banana", se define como:
$$f(x, y) = (1 - x)^2 + 100(y - x^2)^2$$
Su topología presenta un desafío único: un valle estrecho, plano y con forma parabólica. El mínimo global se encuentra en $(1, 1)$.

### 5.1. Por qué falla el Gradiente (Primer Orden)
El método de Descenso de Gradiente utiliza la actualización: $x_{new} = x - \alpha \nabla f(x)$.
En el valle de Rosenbrock, las paredes son muy empinadas (alta pendiente en $y$) pero el fondo es casi plano (baja pendiente a lo largo de la curva).

- El gradiente está dominado por la pendiente de las paredes.
- **Resultado:** El algoritmo "rebota" de una pared a otra (zig-zag) y avanza muy lentamente hacia el mínimo, ignorando la curvatura del valle.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm, solve

# --- 1. DEFINICIÓN DE LA FUNCIÓN DE ROSENBROCK ---
# f(x, y) = (1 - x)^2 + 100*(y - x^2)^2
# Mínimo global en (1, 1)

def f(x):
    return (1 - x[0])**2 + 100 * (x[1] - x[0]**2)**2

def grad_f(x):
    # Derivadas parciales
    df_dx = -2 * (1 - x[0]) - 400 * x[0] * (x[1] - x[0]**2)
    df_dy = 200 * (x[1] - x[0]**2)
    return np.array([df_dx, df_dy])

def hessian_f(x):
    # Matriz de segundas derivadas (Hessiana)
    # Cambia en cada punto (x, y), ya no es constante
    d2f_dx2 = 2 - 400 * x[1] + 1200 * x[0]**2
    d2f_dxdy = -400 * x[0]
    d2f_dy2 = 200
    return np.array([[d2f_dx2, d2f_dxdy],
                     [d2f_dxdy, d2f_dy2]])

# --- 2. ALGORITMOS ---

def gradient_descent(start, lr=0.002, max_iter=2000):
    path = [start]
    x = np.array(start)
    for _ in range(max_iter):
        g = grad_f(x)
        # Nota : Usamos un Learning Rate muy pequeño porque Rosenbrock es "peligrosa"
        x = x - lr * g
        path.append(x)
        if norm(g) < 1e-4: break
    return np.array(path)

def newton_method(start, max_iter=100):
    path = [start]
    x = np.array(start)
    for i in range(max_iter):
        g = grad_f(x)
        H = hessian_f(x)
        
        if norm(g) < 1e-6:
            print(f"Newton convergió en iteración : {i}")
            break
            
        # Paso de Newton : x_new = x - H^(-1) * g
        try:
            direction = solve(H, -g)
            x = x + direction # Learning rate de 1.0 (Full Newton)
            path.append(x)
        except np.linalg.LinAlgError:
            print("Matriz singular encontrada, deteniendo Newton.")
            break
            
    return np.array(path)

# --- 3. EJECUCIÓN ---
# Punto de inicio clásico lejos del óptimo (1,1)
start_point = [-1.5, 2.0]

# Ejecutar Gradiente (necesita muchas iteraciones)
path_gd = gradient_descent(start_point, lr=0.0015, max_iter=5000)

# Ejecutar Newton
path_newton = newton_method(start_point)

print(f"Pasos Gradiente: {len(path_gd)} (Apenas avanza)")
print(f"Pasos Newton: {len(path_newton)} (Llega al destino)")

# --- 4. VISUALIZACIÓN ---
plt.figure(figsize=(12, 8))

# Generar Grid para el mapa de calor
x_vals = np.linspace(-2, 2, 200)
y_vals = np.linspace(-1, 3, 200)
X, Y = np.meshgrid(x_vals, y_vals)
Z = (1 - X)**2 + 100 * (Y - X**2)**2

# Usamos escala logarítmica en los contornos para ver mejor el valle plano
plt.contourf(X, Y, Z, levels=np.logspace(-1, 3, 30), cmap='gray_r', alpha=0.5)
plt.contour(X, Y, Z, levels=np.logspace(-1, 3, 30), colors='black', linewidths=0.3, alpha=0.5)

# Plot Trayectorias
plt.plot(path_gd[:,0], path_gd[:,1], 'r-', linewidth=2, label='Gradiente (Se atasca en el valle)')
plt.plot(path_newton[:,0], path_newton[:,1], 'b-o', linewidth=2, label='Newton (Sigue la curva)')

# Puntos Inicio y Fin
plt.scatter(start_point[0], start_point[1], color='lime', s=100, edgecolors='black', label='Inicio', zorder=10)
plt.scatter(1, 1, color='yellow', marker='*', s=300, edgecolors='black', label='Mínimo Global (1,1)', zorder=10)

plt.title('Rosenbrock ("Banana") Function : Newton vs Gradiente')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True, alpha=0.2)
plt.show()

## 6. La Solución de Newton (Segundo Orden)

El método de Newton introduce la matriz Hessiana $H_f$ en la ecuación de actualización:
$$x_{new} = x - H_f^{-1}(x) \nabla f(x)$$

### 6.1. ¿Qué hace la Hessiana realmente?
Matemáticamente, la Hessiana $H_f$ contiene información sobre la curvatura de la función en todas las direcciones. Al multiplicar el gradiente por la inversa de la Hessiana ($H^{-1}$), ocurren dos fenómenos geométricos vitales:

1. **Re-escalado (Normalización):** La Hessiana detecta que la curvatura en la dirección $y$ es enorme (paredes empinadas) y la curvatura a lo largo del valle es pequeña. La inversa $H^{-1}$ "divide" por esta curvatura. Esto reduce el paso en direcciones empinadas (evitando el rebote) y agranda el paso en direcciones planas (acelerando el avance).
2. **Rotación del Vector:** Ajusta la dirección para seguir la curva óptima en lugar de solo la pendiente local.